In [0]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns

**Defining Neural Net**

In [0]:
class Network(nn.Module):
  
  def __init__(self):
    super(Network, self).__init__()
    
    self.hidden_first    = nn.Linear(20, 21) # the first one is the number of samples of input data
    self.hidden_second   = nn.Linear(21, 17)
    self.hidden_third    = nn.Linear(17, 15)
    self.output          = nn.Linear(15,1)
    
  def forward(self, x):
    x = F.leaky_relu(self.hidden_first(x))
    x = F.leaky_relu(self.hidden_second(x))
    x = F.leaky_relu(self.hidden_third(x))
    x = (self.output(x))
    return x
  

In [0]:

device = torch.device("cuda:0")
device1 = torch.device('cuda:1')
device2 = torch.device('cuda:2')


In [9]:
torch.cuda.is_available()

True

In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [11]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [0]:
# categorical_variables = [ 
#     "MSZoning", "Street", "Alley", "LotShape", "LandContour", "Utilities", "LotConfig", 
#     "LandSlope", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "OverallQual",
#     "OverallCond", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "ExterQual", "ExterCond", "Foundation",
#     "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Heating", "HeatingQC", "CentralAir", "Electrical",
#     "FullBath", "BedroomAbvGr", "KitchenQual",  "KitchenAbvGr", "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu", "GarageType", "GarageFinish", "GarageCars", "GarageQual",
#     "GarageCond", "PavedDrive", "PoolQC", "Fence", "MiscFeature", "SaleType", "SaleCondition"]

outputs_train = train[["Id", "SalePrice"]]

In [0]:
train_data2 = pd.get_dummies(train)
train_data2 = train_data2.drop(["Id"], axis = 1)
train_data2.shape
train_data2 = train_data2.fillna(0)
outputs_real = train_data2["SalePrice"]

In [0]:
train_data3 = train_data2.to_numpy()
mean = np.mean(train_data3, axis = 0)
std = np.std(train_data3, axis = 0)


train_data2 = train_data2.drop(["SalePrice"], axis = 1)

In [0]:
# mean_inps = train_data2.values.mean(axis = 0)
# std_inps = train_data2.values.std(axis = 0)

# mean_ops = outputs_real.values.mean(axis = 0)
# std_ops = outputs_real.values.std(axis = 0)



In [0]:
attra = ["MSSubClass", "LotFrontage", "LotArea", "OverallQual", "YrSold", "EnclosedPorch",  	"OpenPorchSF", "WoodDeckSF", "GarageArea", "GarageYrBlt", 
           	"1stFlrSF", "TotalBsmtSF", "1stFlrSF",  "2ndFlrSF", "BsmtFinSF1", "MasVnrArea", "YearRemodAdd", "YearBuilt", "OverallCond", "OverallQual",  ]

In [0]:
train_data2 = train_data2[attra]

In [0]:

# data4 = np.array(train_data2.values - mean_inps).astype(float)/np.array(std_inps).astype(float)
data4 = train_data2.values
# outputs = (np.array(outputs_real.values - mean_ops).astype(float)/np.array(std_ops) 
outputs = outputs_real.values

In [18]:
data4.shape

(1460, 20)

In [0]:


input = torch.from_numpy(data4).double().to(device)

epochs = 150

target = torch.from_numpy(outputs).double().to(device)
criterion = nn.MSELoss()


In [80]:
net = Network().to(device)
net = net.float()

# target = outputs
batch_size = 20
epochs = 200
optimizer = optim.Adam(net.parameters(), lr = 0.075)

for j in range(epochs):
  i = 0
  while(i < input.shape[0]):
    optimizer.zero_grad()
    net.zero_grad()
    
    batch_input = input[i:i+batch_size]
    batch_output = target[i:i+batch_size]

    
    output = net(batch_input.float())
    loss = criterion(output, target.float())
    
    loss.backward()
    optimizer.step()

    i += batch_size

  print("epoch = ", j, "  _____  loss = ", loss)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([1460])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch =  0   _____  loss =  tensor(6.6547e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  1   _____  loss =  tensor(6.4712e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  2   _____  loss =  tensor(6.3479e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  3   _____  loss =  tensor(6.3205e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  4   _____  loss =  tensor(6.3107e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  5   _____  loss =  tensor(6.3090e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  6   _____  loss =  tensor(6.3084e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  7   _____  loss =  tensor(6.3086e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  8   _____  loss =  tensor(6.3084e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  9   _____  loss =  tensor(6.3091e+09, device='cuda:0', grad_fn=<MseLossBackward>)
epoch =  10   _____  loss =  tensor(6.3111e+09, device='cuda:0', grad_fn=<MseLossBackward>

In [0]:
outs = net(input.float())

**Errors**

- Raw MSE is very high since the values are in range of 10,000, high MSE should not be shocking


- Raw MSE = 6 * 10^9
- MSE (log-outputs) = 0.1595

- The neural net is always converging only to those values no matter what configuration I choose here. So, I am reporting only loss for only one iteration

**Note**
- The above mentioned errors are for the training data


**Possible modifications**
- The loss is converging better and faster when we are trying to estimate log(outputs) than the real values of the outptus. 


**Issues**

- There is a vanishing gradient problem that is being occured in this problem. The gradients are zeros after some point and the error does not decrease beyond a value 

- Proper clustering and pattern analysis methods need to be applied for which there is not sufficient time.

- Dealing with Categorical attributes in the data. Just for convenience we one-hot-encoded the categorical data because of which the dimensionality increased a lot

- Mean-Squared-Error is extremely high for the data

Error Function used = MSELoss

Optimizer           = Adam


In [85]:
print(net)

Network(
  (hidden_first): Linear(in_features=20, out_features=21, bias=True)
  (hidden_second): Linear(in_features=21, out_features=17, bias=True)
  (hidden_third): Linear(in_features=17, out_features=15, bias=True)
  (output): Linear(in_features=15, out_features=1, bias=True)
)
